# Entrando Modelo BERT

In [15]:
import transformers
from tqdm.auto import tqdm
import pandas as pd
import numpy as np
import torch

In [12]:
data = pd.read_csv('D:/Tripleten/datasets/imdb_reviews_small.tsv' ,sep='\t')
corpus = data['review']
tokenizer = transformers.BertTokenizer.from_pretrained('bert-base-uncased')

C:\Users\jm_he\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [13]:
ids_list = []
attention_mask_list = []

max_length = 512

for input_text in corpus[:200]:
    ids = tokenizer.encode(
        input_text.lower(),
        add_special_tokens=True,
        truncation=True,
        max_length=max_length,
    )
    padded = np.array(ids + [0] * (max_length - len(ids)))
    attention_mask = np.where(padded != 0, 1, 0)
    ids_list.append(padded)
    attention_mask_list.append(attention_mask)

In [14]:
# Inicialización de la configuracíon de BertConfig
config = BertConfig.from_pretrained('bert-base-uncased')

#Inicializando el modelo
model = BertModel.from_pretrained('bert-base-uncased')

# tqdm es una biblioteca de Python que te permite agregar barras de progreso a tus bucles o iteraciones.
for i in tqdm(range(int(8e6))):
    pass

  0%|          | 0/8000000 [00:00<?, ?it/s]

In [16]:
batch_size = 100

# creación de una lista vacia de embeddings de reseñas

embeddings = []

for i in tqdm(range(len(ids_list) // batch_size)):

    ids_batch = torch.LongTensor(
        ids_list[batch_size * i : batch_size * (i + 1)]
    )
    attention_mask_batch = torch.LongTensor(
        attention_mask_list[batch_size * i : batch_size * (i + 1)]
    )

    with torch.no_grad():
        batch_embeddings = model(
            ids_batch, attention_mask=attention_mask_batch
        )
    embeddings.append(batch_embeddings[0][:, 0, :].numpy())

  0%|          | 0/2 [00:00<?, ?it/s]

C:\Users\jm_he\AppData\Local\Temp\ipykernel_1312\3713879363.py:9: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ..\torch\csrc\utils\tensor_new.cpp:278.)
  ids_batch = torch.LongTensor(


In [17]:
features = np.concatenate(embeddings)

In [20]:
pd.DataFrame(features)

,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
0,-0.122169,0.092195,-0.249359,-0.055086,-0.228724,-0.551546,-0.114361,0.324784,0.209913,-0.026322,...,-0.164391,-0.184356,0.113958,-0.069024,-0.452524,0.564213,-0.142874,0.145212,0.560320,0.688981
1,-0.549666,-0.151823,-0.416533,-0.066173,-0.344536,-0.474458,0.144660,0.084561,0.363413,-0.095575,...,-0.024756,-0.341043,0.102159,-0.182383,-0.259226,0.807761,0.026817,0.068977,0.819748,0.512458
2,-0.113625,0.067533,-0.326989,0.322551,-0.487812,-0.477263,-0.066992,0.348855,-0.191485,-0.368260,...,-0.085881,-0.240073,0.278587,-0.163645,-0.378652,0.827314,-0.226071,-0.279679,0.667773,0.425052
3,0.367855,-0.305474,0.169522,0.067513,-0.218734,-0.300824,0.067415,0.890659,-0.066575,-0.517766,...,-0.061854,-0.277576,-0.227234,-0.286016,0.106644,0.348105,-0.243855,-0.098266,0.872936,0.404907
4,0.111954,0.005557,0.481005,-0.228786,-0.275876,-0.493747,0.554269,0.407132,0.089381,-0.384155,...,-0.086957,-0.352565,-0.196727,-0.390338,-0.008241,0.261729,-0.164578,-0.314623,0.533540,0.558253
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,0.006953,-0.173385,-0.140757,-0.401268,-0.285115,-0.582246,0.034676,0.883379,0.091907,-0.569235,...,0.026106,-0.688050,0.193886,-0.236663,-0.363652,0.372739,-0.226602,-0.341709,0.557226,0.403866
196,0.320755,0.060361,0.176036,-0.302104,-0.164362,-0.498970,0.080711,1.008229,0.259629,-0.725750,...,-0.048318,-0.280682,0.075472,-0.379457,0.112021,0.135773,-0.040638,0.086089,0.262679,0.498620
197,0.120240,-0.149923,0.086899,-0.340308,-0.211053,-0.274579,0.329264,0.624461,0.226524,-0.202640,...,0.049597,-0.658827,-0.070280,-0.128696,-0.059341,0.252980,-0.199175,-0.176068,0.390736,0.463716
198,-0.236308,-0.127540,0.010173,-0.105859,-0.036008,-0.583316,0.388649,1.011410,0.035036,-0.892944,...,0.081190,-0.585537,0.182939,-0.123127,-0.508081,1.004613,-0.230666,-0.649622,0.730459,-0.051130


In [28]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import accuracy_score

In [30]:
target = np.concatenate(embeddings) #No es real, tengo que obtenerlo de otro lado

In [23]:
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=1/2, random_state=1234)

In [29]:
model = LinearRegression()
model.fit(X_train,y_train)
predict = model.predict(X_test)
accuracy = accuracy_score(y_test,predict)


ValueError: continuous-multioutput is not supported